In [1]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated, List
import operator
from langgraph.checkpoint.sqlite import SqliteSaver
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage, ChatMessage
from dotenv import dotenv_values
import os

memory = SqliteSaver.from_conn_string(":memory:")

In [15]:
config = dotenv_values(".env")

In [1]:
class AgentState(TypedDict):
    task: str
    plan: str
    draft: str
    critique: str
    content: List[str]
    revision_number: int
    max_revisions: int

NameError: name 'TypedDict' is not defined

In [17]:
from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0, openai_api_key=config["OPEN_AI_KEY"])

In [18]:
PLAN_PROMPT = """You are an expert programming code reviewer tasked with writing a high level outline of a Python program. \
Write such an outline for the user provided topic. Give an outline of the code/program along with any relevant notes \
or instructions for the sections."""

In [19]:
WRITER_PROMPT = """You are a programming assistant tasked with writing excellent production-level code.\
Generate the best program/script/code possible for the user's request and the initial outline. \
If the user provides critique, respond with a revised version of your previous attempts. \
Utilize all the information below as needed: 

------

{content}"""

In [20]:
REFLECTION_PROMPT = """You are a code reviewer grading a code/script submission. \
Generate critique and recommendations for the user's submission. \
Provide detailed recommendations, including requests for length, depth, style, etc."""

In [21]:
RESEARCH_PLAN_PROMPT = """You are a programming researcher charged with providing information that can \
be used when writing the following production-level code. Generate a list of search queries that will gather \
any relevant information. Only generate 3 queries max."""


In [22]:
RESEARCH_CRITIQUE_PROMPT = """You are a programming researcher charged with providing information that can \
be used when making any requested revisions (as outlined below). \
Generate a list of search queries that will gather any relevant information. Only generate 3 queries max."""


In [23]:
from langchain_core.pydantic_v1 import BaseModel

class Queries(BaseModel):
    queries: List[str]

In [24]:
from tavily import TavilyClient
import os
tavily = TavilyClient(api_key=os.environ["TAVILY_API_KEY"])

In [25]:
def plan_node(state: AgentState):
    messages = [
        SystemMessage(content=PLAN_PROMPT), 
        HumanMessage(content=state['task'])
    ]
    response = model.invoke(messages)
    return {"plan": response.content}

In [26]:
def research_plan_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_PLAN_PROMPT),
        HumanMessage(content=state['task'])
    ])
    content = state['content'] or []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

In [27]:
def generation_node(state: AgentState):
    content = "\n\n".join(state['content'] or [])
    user_message = HumanMessage(
        content=f"{state['task']}\n\nHere is my plan:\n\n{state['plan']}")
    messages = [
        SystemMessage(
            content=WRITER_PROMPT.format(content=content)
        ),
        user_message
        ]
    response = model.invoke(messages)
    return {
        "draft": response.content, 
        "revision_number": state.get("revision_number", 1) + 1
    }


In [28]:
def reflection_node(state: AgentState):
    messages = [
        SystemMessage(content=REFLECTION_PROMPT), 
        HumanMessage(content=state['draft'])
    ]
    response = model.invoke(messages)
    return {"critique": response.content}

In [29]:
def research_critique_node(state: AgentState):
    queries = model.with_structured_output(Queries).invoke([
        SystemMessage(content=RESEARCH_CRITIQUE_PROMPT),
        HumanMessage(content=state['critique'])
    ])
    content = state['content'] or []
    for q in queries.queries:
        response = tavily.search(query=q, max_results=2)
        for r in response['results']:
            content.append(r['content'])
    return {"content": content}

In [30]:
def should_continue(state):
    if state["revision_number"] > state["max_revisions"]:
        return END
    return "reflect"

In [31]:
builder = StateGraph(AgentState)

In [32]:
builder.add_node("planner", plan_node)
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.add_node("research_plan", research_plan_node)
builder.add_node("research_critique", research_critique_node)

In [33]:
builder.set_entry_point("planner")

In [34]:
builder.add_conditional_edges(
    "generate", 
    should_continue, 
    {END: END, "reflect": "reflect"}
)


In [35]:
builder.add_edge("planner", "research_plan")
builder.add_edge("research_plan", "generate")

builder.add_edge("reflect", "research_critique")
builder.add_edge("research_critique", "generate")

In [36]:
graph = builder.compile(checkpointer=memory)

In [37]:
task = """
Our team needs to update the managed attributes of an entity on Azure Purview using the Purview APIs. Please provide the outline of we can go about in using the APIs along with the actual code.
"""

In [38]:
thread = {"configurable": {"thread_id": "1"}}
for s in graph.stream({
    'task': task,
    "max_revisions": 2,
    "revision_number": 1,
}, thread):
    print(s)

{'planner': {'plan': '**Outline of Python Program to Update Managed Attributes on Azure Purview using Purview APIs:**\n\n1. **Import necessary libraries:**\n   - Import requests library for making HTTP requests to Purview APIs.\n\n2. **Set up authentication:**\n   - Obtain Azure AD token for authentication.\n   - Set headers with the token for API requests.\n\n3. **Define the API endpoint for updating managed attributes:**\n   - Construct the URL for the specific entity\'s managed attributes.\n\n4. **Prepare the data for updating managed attributes:**\n   - Define the new managed attributes data in a dictionary format.\n\n5. **Make a PUT request to update managed attributes:**\n   - Use the requests library to send a PUT request to the API endpoint with the updated managed attributes data.\n   - Handle the response from the API.\n\n6. **Handle errors and exceptions:**\n   - Implement error handling to manage any issues with the API request or response.\n\n7. **Complete the program with